# RAG Retrieve Context

Once the data is uploaded into the vector DB, the next step is connect to an LLM and start retrieving context from the user's query

## Import libraries

In [1]:
from qdrant_client import QdrantClient, models
import requests

import sys

sys.path.append("..")

from rag_llm_energy_expert.search.searchers_auxiliars import process_query, process_query_results
from rag_llm_energy_expert.search.searchers import semantic_search
from rag_llm_energy_expert.credentials import get_qdrant_config, get_gcp_config

In [2]:
qdrant_config = get_qdrant_config()
gcp_config = get_gcp_config()

Vector DB connection

In [3]:
qdrant_client = QdrantClient(url = qdrant_config.URL, api_key = qdrant_config.API_KEY.get_secret_value())

Query to obtain the most relevant documents

In [4]:
query = "Se establecen mecanismos legales efectivos para prevenir"

Embedding of the query

In [5]:
payload = {
    "text":query,
}


headers = {"Authorization": f"Bearer {gcp_config.EMBEDDING_SERVICE_IDTOKEN.get_secret_value()}"}

response = requests.post(url = gcp_config.EMBEDDING_SERVICE_URL + gcp_config.EMBED_TEXT_ENDPOINT, json=payload, headers = headers)

In [6]:
response.text

'{"chunks":[{"vector_id":"8702880b-c394-4ece-ac3d-df22a0302750","vector":[-0.10738659650087357,0.2562285363674164,-0.4657200276851654,-0.06869310885667801,0.07532059401273727,0.38594698905944824,-0.09326693415641785,0.1372416913509369,-0.008625430054962635,0.4161311388015747,0.265689492225647,0.35646137595176697,0.14466364681720734,-0.04870103672146797,-0.02861391194164753,-0.05373667553067207,-0.06363384425640106,0.2452787607908249,-0.09690675884485245,0.12638919055461884,0.41744673252105713,0.024091007187962532,0.007919087074697018,0.014006945304572582,-0.5610426068305969,0.12132805585861206,-0.2948296070098877,-0.14888982474803925,0.14998102188110352,-0.10311264544725418,-0.12275881320238113,-0.22059790790081024,-0.21566633880138397,0.05584513023495674,-0.20673984289169312,-0.5407423377037048,0.18498051166534424,0.2138666957616806,0.008598199114203453,-0.17691625654697418,-0.05485868453979492,0.04597483202815056,-0.21201592683792114,0.08601651340723038,-0.19417540729045868,0.0651322

In [7]:
embeddings = response.json()["chunks"]

Getting the vectors generated

In [8]:
vectors = [chunk["vector"] for chunk in embeddings]

In [9]:
len(vectors[0])

384

Preparing the vectors for semantic search

In [10]:
search_queries = [models.QueryRequest(
    query = vector, 
    with_payload = True,
    with_vector = False,
    limit = 5) for vector in vectors]

search_queries

[QueryRequest(shard_key=None, prefetch=None, query=[-0.10738659650087357, 0.2562285363674164, -0.4657200276851654, -0.06869310885667801, 0.07532059401273727, 0.38594698905944824, -0.09326693415641785, 0.1372416913509369, -0.008625430054962635, 0.4161311388015747, 0.265689492225647, 0.35646137595176697, 0.14466364681720734, -0.04870103672146797, -0.02861391194164753, -0.05373667553067207, -0.06363384425640106, 0.2452787607908249, -0.09690675884485245, 0.12638919055461884, 0.41744673252105713, 0.024091007187962532, 0.007919087074697018, 0.014006945304572582, -0.5610426068305969, 0.12132805585861206, -0.2948296070098877, -0.14888982474803925, 0.14998102188110352, -0.10311264544725418, -0.12275881320238113, -0.22059790790081024, -0.21566633880138397, 0.05584513023495674, -0.20673984289169312, -0.5407423377037048, 0.18498051166534424, 0.2138666957616806, 0.008598199114203453, -0.17691625654697418, -0.05485868453979492, 0.04597483202815056, -0.21201592683792114, 0.08601651340723038, -0.19417

Making the semantic search

In [11]:
collection_name = qdrant_config.COLLECTION_NAME + qdrant_config.COLLECTION_VERSION

results = qdrant_client.query_batch_points(
    collection_name = collection_name,
    requests= search_queries,
)

Obtaining the text of each document retrieved

In [12]:
full_text = ""

for query_response in results:

    query_response_points = query_response.points

    for point in query_response_points:

        full_text += point.payload["text"] + "\n\n"

In [13]:
print(full_text)

 reglamento de la Ley correspondiente.
• Con la finalidad de salvaguardar los intereses y derechos de las comunidades en las que se de-
sarrollen proyectos del sector hidrocarburos, el Artículo 120 establece que deberán llevarse 
a cabo los procedimientos de consulta, en los cuales podrán participar las Empresas Produc-
tivas del Estado y sus subsidiarias y filiales, así como partic


• En el Artículo 27 Constitucional: 
	
Se establece que la planeación y el control del Sistema Eléctrico Nacional, así como el ser-
vicio público de transmisión y distribución de electricidad son áreas exclusivas del Estado. 
Se mantiene la prohibición expresa de otorgar concesiones en estas áreas y se permite 
que el Estado celebre contratos con particulares. De

ulares, conforme a la normatividad 
aplicable.
• En lo relativo al uso y ocupación superficial, la legislación secundaria establece el mecanismo 
basado en principios de transparencia y equidad para definir los términos y las condiciones 
genera

## Semantic Search with the generated functions

In [14]:
query = "En el nuevo modelo, cómo se considera a Pemex?"

Obtaining the vectors with the QueryRequest format

In [15]:
search_queries = process_query(
    query=query,
    embedding_model_name=None,
    chunk_overlap=0,
    documents_limit=3
)

2025-04-16 22:55:22.274 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:37 - Preprocessing query...
2025-04-16 22:55:22.276 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:74 - Generating embeddings...
2025-04-16 22:55:26.405 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:87 - Embeddings generated successfully
2025-04-16 22:55:26.405 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:95 - Preparing embeddings for vector search
2025-04-16 22:55:26.405 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:106 - Query preprocessed successfully


In [16]:
collection_name = qdrant_config.COLLECTION_NAME + qdrant_config.COLLECTION_VERSION

results = qdrant_client.query_batch_points(
    collection_name = collection_name,
    requests= search_queries,
)

Obtaining the text retrieved from semantic search

In [17]:
print(process_query_results(results))

2025-04-16 22:55:26.520 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query_results:120 - Processing query results...
2025-04-16 22:55:26.524 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query_results:131 - Query results processed



adjudicado el contrato. Cuando una asignación migre a un contrato podrá llevarse a cabo una 
asociación entre Pemex y un partícular, y la CNH realizará una licitación para elegir al socio 
(incluyendo las asignaciones de la Ronda Cero).
• El modelo propuesto también considera que Pemex podrá migrar a la nueva modalidad de 
contratación

 Pemex en la industria petrolera. Mediante 
la “Ronda Cero”, Pemex podrá elegir aquellos campos en producción y aquellas áreas en ex-
ploración que tengan interés en operar y donde demuestre tener capacidad técnica, financie-
ra y de ejecución para desarrollarlos en forma eficiente y competitiva y podrá migrarlas hacia 
un esqu

 pensiones y jubilaciones 
de PEMEX y CFE, sujeto a que acuerden con sus trabajadores un nuevo régimen de pensiones 
que reduzca esos pasivos y la Auditoria Superior de la Federación audite la evolución de di-
chos pasivos.
• Los Consejos de Administración de ambas empresas tendrán una nueva estructura organi-
zacional y se enc

## Using the semantic search function

In [33]:
query = "Cuáles son los beneficios de la reforma energética?"

In [17]:
collection_name = qdrant_config.COLLECTION_NAME + qdrant_config.COLLECTION_VERSION

In [34]:
context = semantic_search(
    query = query,
    collection_name=collection_name,
    chunk_overlap=None,
    embedding_model_name='sentence-transformers/LaBSE',
    documents_limit=15
)

print(context)

2025-04-20 19:00:07.682 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:37 - Preprocessing query...
2025-04-20 19:00:07.684 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:74 - Generating embeddings...
2025-04-20 19:00:12.318 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:87 - Embeddings generated successfully
2025-04-20 19:00:12.318 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:95 - Preparing embeddings for vector search
2025-04-20 19:00:12.318 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:106 - Query preprocessed successfully
2025-04-20 19:00:12.682 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query_results:120 - Processing query results...
2025-04-20 19:00:12.682 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query_results:131 - Query results processed


## I. Introducción  
La Reforma Energética es un paso decidido rumbo a la modernización del sector energético de
nuestro país, sin privatizar las empresas públicas dedicadas a la producción y al aprovechamiento de los hidrocarburos y de la electricidad. La Reforma Energética, tanto constitucional como a
nivel legistlación secundarias, surge del estudio y valoración de las distintas iniciativas presentadas por los partidos políticos representados en el Congreso.

## VIII. Ocupación de la superficie, sustentabilidad y enfoque de derechos  
### La Reforma Energética Constitucional  
- La Reforma Energética dispone en el Artículo 8 Transitorio que las actividades de exploración  
y extracción del petróleo y de los demás hidrocarburos, así como el servicio público de trans

sustentable el gran potencial energético que tiene nuestro país.  
La Reforma Constitucional, aprobada por el Congreso de la Unión y publicada en el Diario Oficial
de la Federación en diciembre de 2013, representó el pri